# Human-in-the-Loop Arbeidsflyt med Microsoft Agent Framework

## 🎯 Læringsmål

I denne notatboken lærer du hvordan du implementerer **human-in-the-loop** arbeidsflyter ved hjelp av Microsoft Agent Frameworks `RequestInfoExecutor`. Dette kraftige mønsteret lar deg pause AI-arbeidsflyter for å samle inn menneskelig input, gjøre agentene interaktive og gi mennesker kontroll over kritiske beslutninger.

## 🔄 Hva er Human-in-the-Loop?

**Human-in-the-loop (HITL)** er et designmønster der AI-agenter pauser utførelsen for å be om menneskelig input før de fortsetter. Dette er viktig for:

- ✅ **Kritiske beslutninger** - Få menneskelig godkjenning før viktige handlinger utføres
- ✅ **Tvetydige situasjoner** - La mennesker avklare når AI er usikker
- ✅ **Brukerpreferanser** - Be brukere velge mellom flere alternativer
- ✅ **Samsvar og sikkerhet** - Sikre menneskelig tilsyn for regulerte operasjoner
- ✅ **Interaktive opplevelser** - Bygg samtaleagenter som reagerer på brukerinput

## 🏗️ Hvordan det fungerer i Microsoft Agent Framework

Rammeverket tilbyr tre nøkkelkomponenter for HITL:

1. **`RequestInfoExecutor`** - En spesiell executor som pauser arbeidsflyten og sender ut en `RequestInfoEvent`
2. **`RequestInfoMessage`** - Basisklasse for typede forespørselsdata som sendes til mennesker
3. **`RequestResponse`** - Knytter menneskelige svar til de opprinnelige forespørslene ved hjelp av `request_id`

**Arbeidsflytmønster:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Vårt eksempel: Hotellbestilling med brukerbekreftelse

Vi bygger videre på den betingede arbeidsflyten ved å legge til menneskelig bekreftelse **før** vi foreslår alternative destinasjoner:

1. Brukeren forespør en destinasjon (f.eks. "Paris")
2. `availability_agent` sjekker om rom er tilgjengelige
3. **Hvis ingen rom** → `confirmation_agent` spør "Vil du se alternativer?"
4. Arbeidsflyten **pauser** ved bruk av `RequestInfoExecutor`
5. **Mennesket svarer** "ja" eller "nei" via konsollinput
6. `decision_manager` styrer basert på svaret:
   - **Ja** → Vis alternative destinasjoner
   - **Nei** → Avbryt bestillingsforespørselen
7. Viser endelig resultat

Dette demonstrerer hvordan man gir brukere kontroll over agentens forslag!

---

La oss komme i gang! 🚀


## Steg 1: Importer nødvendige biblioteker

Vi importerer de standard komponentene i Agent Framework pluss **spesifikke klasser for menneskelig interaksjon**:
- `RequestInfoExecutor` - Utfører som stopper arbeidsflyten for menneskelig input
- `RequestInfoEvent` - Hendelse som utløses når menneskelig input blir forespurt
- `RequestInfoMessage` - Grunnklasse for typede forespørselsdata
- `RequestResponse` - Knytter menneskelige svar til forespørsler
- `WorkflowOutputEvent` - Hendelse for å oppdage utdata fra arbeidsflyten


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Steg 2: Definer Pydantic-modeller for strukturerte utdata

Disse modellene definerer **skjemaet** som agenter vil returnere. Vi beholder alle modeller fra den betingede arbeidsflyten og legger til:

**Nytt for menneskelig involvering:**
- `HumanFeedbackRequest` - Underklasse av `RequestInfoMessage` som definerer forespørselspayloaden som sendes til mennesker
  - Inneholder `prompt` (spørsmål som skal stilles) og `destination` (kontekst om den utilgjengelige byen)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Steg 3: Lag verktøyet for hotellbestilling

Samme verktøy som i den betingede arbeidsflyten - sjekker om rom er tilgjengelige på destinasjonen.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Steg 4: Definer betingelsesfunksjoner for ruting

Vi trenger **fire betingelsesfunksjoner** for vår human-in-the-loop arbeidsflyt:

**Fra betinget arbeidsflyt:**
1. `has_availability_condition` - Ruter når hoteller ER tilgjengelige
2. `no_availability_condition` - Ruter når hoteller IKKE er tilgjengelige

**Nytt for human-in-the-loop:**
3. `user_wants_alternatives_condition` - Ruter når brukeren sier "ja" til alternativer
4. `user_declines_alternatives_condition` - Ruter når brukeren sier "nei" til alternativer


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Steg 5: Opprett Decision Manager Executor

Dette er **kjernen i human-in-the-loop-mønsteret**! `DecisionManager` er en tilpasset `Executor` som:

1. **Mottar tilbakemeldinger fra mennesker** via `RequestResponse`-objekter
2. **Behandler brukerens beslutning** (ja/nei)
3. **Styrer arbeidsflyten** ved å sende meldinger til riktige agenter

Nøkkelfunksjoner:
- Bruker `@handler`-dekoratoren for å eksponere metoder som arbeidsflytsteg
- Mottar `RequestResponse[HumanFeedbackRequest, str]` som inneholder både den opprinnelige forespørselen og brukerens svar
- Gir enkle "ja" eller "nei"-meldinger som utløser våre betingelsesfunksjoner


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Steg 6: Opprett egendefinert visningsutfører

Samme visningsutfører fra betinget arbeidsflyt - gir endelige resultater som arbeidsflytens output.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Steg 7: Last inn miljøvariabler

Konfigurer LLM-klienten (GitHub Models, Azure OpenAI eller OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Steg 8: Opprett AI-agenter og eksekutorer

Vi oppretter **seks arbeidsflytkomponenter**:

**Agenter (pakket inn i AgentExecutor):**
1. **availability_agent** - Sjekker hotelltilgjengelighet ved hjelp av verktøyet
2. **confirmation_agent** - 🆕 Forbereder forespørsel om bekreftelse fra menneske
3. **alternative_agent** - Foreslår alternative byer (når brukeren sier ja)
4. **booking_agent** - Oppmuntrer til booking (når rom er tilgjengelige)
5. **cancellation_agent** - 🆕 Håndterer avbestillingsmelding (når brukeren sier nei)

**Spesielle eksekutorer:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` som pauser arbeidsflyten for menneskelig input
7. **decision_manager** - 🆕 Tilpasset eksekutor som ruter basert på menneskelig respons (allerede definert ovenfor)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Steg 9: Bygg arbeidsflyten med menneskelig involvering

Nå konstruerer vi arbeidsflytens graf med **betinget ruting** inkludert veien for menneskelig involvering:

**Struktur for arbeidsflyt:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Viktige forbindelser:**
- `availability_agent → confirmation_agent` (når det ikke er rom)
- `confirmation_agent → prepare_human_request` (transformasjonstype)
- `prepare_human_request → request_info_executor` (pause for menneskelig involvering)
- `request_info_executor → decision_manager` (alltid - gir RequestResponse)
- `decision_manager → alternative_agent` (når brukeren sier "ja")
- `decision_manager → cancellation_agent` (når brukeren sier "nei")
- `availability_agent → booking_agent` (når rom er tilgjengelige)
- Alle veier ender ved `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Steg 10: Kjør testtilfelle 1 - By UTEN tilgjengelighet (Paris med menneskelig bekreftelse)

Denne testen demonstrerer **hele menneske-i-løkken-syklusen**:

1. Forespør hotell i Paris
2. availability_agent sjekker → Ingen rom
3. confirmation_agent lager et spørsmål rettet mot mennesker
4. request_info_executor **pauser arbeidsflyten** og sender ut `RequestInfoEvent`
5. **Applikasjonen oppdager hendelsen og ber brukeren om input i konsollen**
6. Brukeren skriver "ja" eller "nei"
7. Applikasjonen sender svar via `send_responses_streaming()`
8. decision_manager ruter basert på svaret
9. Endelig resultat vises

**Nøkkelmønster:**
- Bruk `workflow.run_stream()` for første iterasjon
- Bruk `workflow.send_responses_streaming(pending_responses)` for påfølgende iterasjoner
- Lytt etter `RequestInfoEvent` for å oppdage når menneskelig input er nødvendig
- Lytt etter `WorkflowOutputEvent` for å fange opp endelige resultater


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Steg 11: Kjør testtilfelle 2 - By MED tilgjengelighet (Stockholm - Ingen menneskelig input nødvendig)

Denne testen demonstrerer den **direkte veien** når rom er tilgjengelige:

1. Forespør hotell i Stockholm
2. availability_agent sjekker → Rom tilgjengelige ✅
3. booking_agent foreslår booking
4. display_result viser bekreftelse
5. **Ingen menneskelig input nødvendig!**

Arbeidsflyten omgår helt veien med menneskelig involvering når rom er tilgjengelige.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Viktige punkter og beste praksis for menneskelig involvering

### ✅ Hva du har lært:

#### 1. **RequestInfoExecutor-mønster**
Mønsteret for menneskelig involvering i Microsoft Agent Framework bruker tre nøkkelkomponenter:
- `RequestInfoExecutor` - Pauser arbeidsflyten og sender ut hendelser
- `RequestInfoMessage` - Basisklasse for typede forespørselsdata (subklass denne!)
- `RequestResponse` - Knytter menneskelige svar til opprinnelige forespørsler

**Viktig forståelse:**
- `RequestInfoExecutor` samler IKKE inn input selv - den pauser kun arbeidsflyten
- Applikasjonskoden din må lytte etter `RequestInfoEvent` og samle inn input
- Du må kalle `send_responses_streaming()` med en dict som kobler `request_id` til brukerens svar

#### 2. **Streaming Execution-mønster**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Hendelsesdrevet arkitektur**
Lytt etter spesifikke hendelser for å kontrollere arbeidsflyten:
- `RequestInfoEvent` - Menneskelig input er nødvendig (arbeidsflyten pauset)
- `WorkflowOutputEvent` - Endelig resultat er tilgjengelig (arbeidsflyten fullført)
- `WorkflowStatusEvent` - Tilstandsforandringer (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, osv.)

#### 4. **Egendefinerte executors med @handler**
`DecisionManager` viser hvordan man lager executors som:
- Bruker `@handler`-dekoratoren for å eksponere metoder som arbeidsflytsteg
- Mottar typede meldinger (f.eks. `RequestResponse[HumanFeedbackRequest, str]`)
- Ruter arbeidsflyten ved å sende meldinger til andre executors
- Får tilgang til kontekst via `WorkflowContext`

#### 5. **Betinget ruting med menneskelige beslutninger**
Du kan lage funksjoner som evaluerer menneskelige svar:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Virkelige applikasjoner:

1. **Godkjenningsarbeidsflyter**
   - Få ledergodkjenning før behandling av utgiftsrapporter
   - Krev menneskelig gjennomgang før sending av automatiserte e-poster
   - Bekreft transaksjoner med høy verdi før utførelse

2. **Innholdsmoderering**
   - Flagge tvilsomt innhold for menneskelig gjennomgang
   - Be moderatorer ta endelige avgjørelser i grensesaker
   - Eskalere til mennesker når AI-sikkerheten er lav

3. **Kundeservice**
   - La AI håndtere rutinemessige spørsmål automatisk
   - Eskalere komplekse saker til menneskelige agenter
   - Spør kunden om de vil snakke med et menneske

4. **Databehandling**
   - Be mennesker løse tvetydige dataoppføringer
   - Bekreft AI-tolkninger av uklare dokumenter
   - La brukere velge mellom flere gyldige tolkninger

5. **Sikkerhetskritiske systemer**
   - Krev menneskelig bekreftelse før irreversible handlinger
   - Få godkjenning før tilgang til sensitiv data
   - Bekreft beslutninger i regulerte bransjer (helsevesen, finans)

6. **Interaktive agenter**
   - Bygg samtaleboter som stiller oppfølgingsspørsmål
   - Lag veivisere som guider brukere gjennom komplekse prosesser
   - Design agenter som samarbeider med mennesker steg-for-steg

### 🔄 Sammenligning: Med vs uten menneskelig involvering

| Funksjon | Betinget arbeidsflyt | Arbeidsflyt med menneskelig involvering |
|----------|-----------------------|-----------------------------------------|
| **Utførelse** | Enkel `workflow.run()` | Loop med `run_stream()` + `send_responses_streaming()` |
| **Brukerinput** | Ingen (fullt automatisert) | Interaktive spørsmål via `input()` eller UI |
| **Komponenter** | Agenter + Executors | + RequestInfoExecutor + DecisionManager |
| **Hendelser** | Kun AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, osv. |
| **Pausing** | Ingen pausing | Arbeidsflyten pauser ved RequestInfoExecutor |
| **Menneskelig kontroll** | Ingen menneskelig kontroll | Mennesker tar viktige beslutninger |
| **Bruksområde** | Automatisering | Samarbeid og tilsyn |

### 🚀 Avanserte mønstre:

#### Flere menneskelige beslutningspunkter
Du kan ha flere `RequestInfoExecutor`-noder i samme arbeidsflyt:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Timeout-håndtering
Implementer tidsfrister for menneskelige svar:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrering med rik UI
I stedet for `input()`, integrer med web UI, Slack, Teams, osv.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Betinget menneskelig involvering
Be kun om menneskelig input i spesifikke situasjoner:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Beste praksis:

1. **Alltid subklass RequestInfoMessage**
   - Gir typesikkerhet og validering
   - Muliggjør rik kontekst for UI-rendering
   - Klargjør intensjonen bak hver forespørselstype

2. **Bruk beskrivende spørsmål**
   - Inkluder kontekst om hva du spør om
   - Forklar konsekvensene av hvert valg
   - Hold spørsmålene enkle og tydelige

3. **Håndter uventet input**
   - Valider brukerens svar
   - Gi standardverdier for ugyldig input
   - Gi tydelige feilmeldinger

4. **Spor forespørsels-IDer**
   - Bruk korrelasjonen mellom request_id og svar
   - Ikke prøv å administrere tilstanden manuelt

5. **Design for ikke-blokkerende prosesser**
   - Ikke blokker tråder som venter på input
   - Bruk asynkrone mønstre gjennomgående
   - Støtt samtidige arbeidsflytinstanser

### 📚 Relaterte konsepter:

- **Agent Middleware** - Avskjær agentkall (forrige notatbok)
- **Arbeidsflyt tilstandshåndtering** - Bevar arbeidsflytens tilstand mellom kjøringer
- **Samarbeid mellom flere agenter** - Kombiner menneskelig involvering med agentteam
- **Hendelsesdrevet arkitektur** - Bygg reaktive systemer med hendelser

---

### 🎓 Gratulerer!

Du har mestret arbeidsflyter med menneskelig involvering i Microsoft Agent Framework! Du vet nå hvordan du:
- ✅ Pauser arbeidsflyter for å samle menneskelig input
- ✅ Bruker RequestInfoExecutor og RequestInfoMessage
- ✅ Håndterer streamingutførelse med hendelser
- ✅ Lager egendefinerte executors med @handler
- ✅ Ruter arbeidsflyter basert på menneskelige beslutninger
- ✅ Bygger interaktive AI-agenter som samarbeider med mennesker

**Dette er et kritisk mønster for å bygge pålitelige, kontrollerbare AI-systemer!** 🚀



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
